# Mesh2d examples

This notebook will illustrate the usage of the mesh2d module. How to create a grid, add data to it and show how to fill the ghost cells.

In [1]:
## First we need to import all the modules that we will need.

import mesh2d.mesh as mesh
import mesh2d.boundaryconditions as bnd
import numpy as np

## Setup a Grid with Variables

When creating a grid, there are a few core classes that you need to deal with:

- *Grid2D*: Holds the size of the grid (counted in zones) and the physical coordinate information, including the coordinates of cell edges and centers.
- *BC*: A container class that holds the type of boundary condition on each domain edge.
- *ExtendedArray*: Extension class to the ordinary numpy array to ease up the use of stencil methods that are often found in finite volume methods.
- *CellCenterData2D*: Holds that data that lives on a grid.

Start by creating a Grid2D object, with 2x6 cells in the x- and y-direction respectively, and 3 ghost cells in each direction.

In [2]:
grid = mesh.Grid2D(2, 6, ng = 3)

In [3]:
print(grid)

2D grid with: 2 x-zones, 6 y-zones, 3 ghostcells


In [4]:
help(grid)

Help on Grid2D in module mesh2d.mesh object:

class Grid2D(builtins.object)
 |  Grid2D(nx, ny, ng=1, xmin=0.0, xmax=1.0, ymin=0.0, ymax=1.0)
 |  
 |  The Grid2D class will contain the coordinate information of the data
 |  that will live at various centerings.
 |  
 |  A 1 dimensional representation of a single row of data looks like:
 |     |     |       |     //     |     |       |     |     //     |       |     |
 |     +--*--+- ... -+--*--//--*--+--*--+- ... -+--*--+--*--//--*--+- ... -+--*--+
 |        0          ng-1    ng   ng+1         ... ng+nx-1 ng+nx      2ng+nx-1
 |                          ilo                      ihi
 |     |<- ng ghostcells ->|<-- nx interior (data) zones -->|<- ng ghostcells ->|
 |  Where * mark the data locations and // "boundaries" between ghost and data cells.
 |  + or | symbols show edges/faces between adjacent cells.
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Check whether two different Grid2D object are equivalent in terms

Define the boundary conditions that will be used to apply to the data that will end up living on the grid.

In [5]:
bc = bnd.BC(xlb = "outflow", xrb = "reflect-odd",
            ylb = "periodic", yrb = "periodic")

Create a dataset that lives on the grid, add some variables to it. And do not forget to register the boundary conditions.

In [6]:
data = mesh.CellCenterData2D(grid)

In [7]:
data.register_var("density")
data.register_var("pressure")
data.register_bcs(bc)

In [8]:
data.create()

In [9]:
print(data)

Cell centered data: 
 2 x-zones, 6 y-zones, 3 ghostcells 
  Number of variables = 2
  Variable names:
      density          min =    0.0000000000,    max =    0.0000000000
      pressure         min =    0.0000000000,    max =    0.0000000000
BCs: -x = outflow     , +x = reflect-odd , -y = periodic    , +y = periodic    


## Working with the data

Filling the grid with data is done in two easy steps. First use the get_var() method of the data object, which returns an ExtendedArray object which contains all the methods you need to access and view the data. Here we use dens.valid() to access the "valid" region of the density field in the data object and fill it with some random numbers.

In [10]:
dens = data.get_var("density")
dens.valid()[:, :] = np.random.rand(grid.nx, grid.ny)

We can see what the field now looks like by calling the pprint function, pprint is short for prettyprint. Clearly we only filled in the valid region only, the ghost cells, displayed in red, are still empty.

In [11]:
dens.pprint()

         0         0         0         0         0         0         0         0
         0         0         0         0         0         0         0         0
         0         0         0         0         0         0         0         0
         0         0         0   0.24029   0.58908         0         0         0
         0         0         0   0.70382  0.057967         0         0         0
         0         0         0   0.52607   0.76328         0         0         0
         0         0         0   0.64854   0.68743         0         0         0
         0         0         0   0.67896   0.34453         0         0         0
         0         0         0   0.17459   0.77377         0         0         0
         0         0         0         0         0         0         0         0
         0         0         0         0         0         0         0         0
         0         0         0         0         0         0         0         0

         ^ y
         |
   

We can repeat the same procedure for the second field in the datacube, being the pressure in our case.

In [12]:
pres = data.get_var("pressure")
pres.valid()[:, :] = np.random.rand(grid.nx, grid.ny)

In [13]:
pres.pprint()

         0         0         0         0         0         0         0         0
         0         0         0         0         0         0         0         0
         0         0         0         0         0         0         0         0
         0         0         0   0.59371   0.78496         0         0         0
         0         0         0   0.70269   0.91708         0         0         0
         0         0         0   0.76951   0.69858         0         0         0
         0         0         0   0.24478   0.40746         0         0         0
         0         0         0    0.5503   0.38603         0         0         0
         0         0         0   0.30456   0.30955         0         0         0
         0         0         0         0         0         0         0         0
         0         0         0         0         0         0         0         0
         0         0         0         0         0         0         0         0

         ^ y
         |
   

Then we fill in the ghost cells according to our boundary conditions using the fill_BC() method of the data object. Using pprint() we can demonstrate that the boundaries are indeed filled now, and are displayed in red.

In [14]:
data.fill_BC()

In [15]:
dens.pprint()

   0.64854   0.64854   0.64854   0.64854   0.68743  -0.68743  -0.64854  -0.64854
   0.67896   0.67896   0.67896   0.67896   0.34453  -0.34453  -0.67896  -0.67896
   0.17459   0.17459   0.17459   0.17459   0.77377  -0.77377  -0.17459  -0.17459
   0.24029   0.24029   0.24029   0.24029   0.58908  -0.58908  -0.24029  -0.24029
   0.70382   0.70382   0.70382   0.70382  0.057967 -0.057967  -0.70382  -0.70382
   0.52607   0.52607   0.52607   0.52607   0.76328  -0.76328  -0.52607  -0.52607
   0.64854   0.64854   0.64854   0.64854   0.68743  -0.68743  -0.64854  -0.64854
   0.67896   0.67896   0.67896   0.67896   0.34453  -0.34453  -0.67896  -0.67896
   0.17459   0.17459   0.17459   0.17459   0.77377  -0.77377  -0.17459  -0.17459
   0.24029   0.24029   0.24029   0.24029   0.58908  -0.58908  -0.24029  -0.24029
   0.70382   0.70382   0.70382   0.70382  0.057967 -0.057967  -0.70382  -0.70382
   0.52607   0.52607   0.52607   0.52607   0.76328  -0.76328  -0.52607  -0.52607

         ^ y
         |
   

In [16]:
pres.pprint()

   0.24478   0.24478   0.24478   0.24478   0.40746  -0.40746  -0.24478  -0.24478
    0.5503    0.5503    0.5503    0.5503   0.38603  -0.38603   -0.5503   -0.5503
   0.30456   0.30456   0.30456   0.30456   0.30955  -0.30955  -0.30456  -0.30456
   0.59371   0.59371   0.59371   0.59371   0.78496  -0.78496  -0.59371  -0.59371
   0.70269   0.70269   0.70269   0.70269   0.91708  -0.91708  -0.70269  -0.70269
   0.76951   0.76951   0.76951   0.76951   0.69858  -0.69858  -0.76951  -0.76951
   0.24478   0.24478   0.24478   0.24478   0.40746  -0.40746  -0.24478  -0.24478
    0.5503    0.5503    0.5503    0.5503   0.38603  -0.38603   -0.5503   -0.5503
   0.30456   0.30456   0.30456   0.30456   0.30955  -0.30955  -0.30456  -0.30456
   0.59371   0.59371   0.59371   0.59371   0.78496  -0.78496  -0.59371  -0.59371
   0.70269   0.70269   0.70269   0.70269   0.91708  -0.91708  -0.70269  -0.70269
   0.76951   0.76951   0.76951   0.76951   0.69858  -0.69858  -0.76951  -0.76951

         ^ y
         |
   

## Coarser and Finer grids

Using our data fields, we can also easily create fields that are either coarser or finer grids. Coarsening can happen by a factor 2 or 4, while prolonging of the grid can only happen in increments of 2. Let's define a new grid and see how these operations work. Starting with an 8x8 grid and 2 ghostcells. Setting up the grid with some data on it first:

In [35]:
grid = mesh.Grid2D(8, 8, ng = 2)
print(grid)

2D grid with: 8 x-zones, 8 y-zones, 2 ghostcells


In [36]:
bc = bnd.BC(xlb = "outflow", xrb = "reflect-odd",
            ylb = "periodic", yrb = "periodic")

In [37]:
newdata = mesh.CellCenterData2D(grid)

newdata.register_var("density")
newdata.register_var("pressure")
newdata.register_bcs(bc)
newdata.create()

In [38]:
dens = newdata.get_var("density")
dens.valid()[:, :] = np.random.rand(grid.nx, grid.ny)

newdata.fill_BC()

dens.pprint(fmt = "%7.2g")

    0.6    0.6    0.6  0.043   0.68   0.19    0.2    0.9  0.059   0.27  -0.27 -0.059
    0.7    0.7    0.7   0.17   0.43   0.59   0.37   0.25   0.82   0.43  -0.43  -0.82
   0.82   0.82   0.82   0.19   0.95   0.68    0.9   0.64   0.76   0.65  -0.65  -0.76
   0.84   0.84   0.84   0.67  0.081   0.99   0.15   0.75   0.77   0.51  -0.51  -0.77
   0.23   0.23   0.23   0.92   0.11   0.17   0.78   0.79   0.71   0.66  -0.66  -0.71
  0.042  0.042  0.042   0.35   0.81   0.53    0.2   0.19    0.2   0.26  -0.26   -0.2
   0.84   0.84   0.84   0.45   0.44    0.2   0.28    0.8   0.16   0.69  -0.69  -0.16
   0.81   0.81   0.81   0.99   0.45   0.83   0.92   0.85   0.48   0.28  -0.28  -0.48
    0.6    0.6    0.6  0.043   0.68   0.19    0.2    0.9  0.059   0.27  -0.27 -0.059
    0.7    0.7    0.7   0.17   0.43   0.59   0.37   0.25   0.82   0.43  -0.43  -0.82
   0.82   0.82   0.82   0.19   0.95   0.68    0.9   0.64   0.76   0.65  -0.65  -0.76
   0.84   0.84   0.84   0.67  0.081   0.99   0.15   0.75   0.77  

Now let's coarsen this grid by a factor of 4:

In [39]:
coarse = newdata.restrict("density", N = 4)

As expected, we retrieve a grid that is coarsened by a factor 4 in both directions, while retaining the number of ghostcells.

In [40]:
coarse.pprint()

         0         0         0         0         0         0
         0         0         0         0         0         0
         0         0   0.52457   0.55667         0         0
         0         0   0.52571   0.48422         0         0
         0         0         0         0         0         0
         0         0         0         0         0         0

         ^ y
         |
         +---> x
        


Let's also try to make the resolution a bit finer. Note: the pretty printing of the array is starting to look a bit messy now, since it is only meant to work on the smaller arrays.

In [46]:
fine = newdata.prolong("density")
fine.pprint(fmt = "%5.2g")

    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0 0.89  1.10.079 0.52  0.9  1.1 0.65  1.1  0.8  1.1 0.55 0.97 0.79  1.1 0.75 0.78    0    0
    0    0  0.5 0.75-0.13 0.31 0.77 0.99 0.25 0.71 0.67    1 0.31 0.72  0.4 0.72 0.52 0.54    0    0
    0    0 0.78  1.2  0.7 0.930.0054 0.42  1.1  1.1 0.15 0.58 0.81    1  0.8  1.1 0.64 0.71    0    0
    0    0 0.52  0.9 0.42 0.65-0.26 0.16 0.85 0.91-0.28 0.16 0.45 0.68 0.43 0.75 0.31 0.38    0    0
    0    0  0.2 0.49    1  1.10.087 0.36 0.24 0.47 0.71 0.95 0.72  1.1 0.64    1 0.75 0.77    0    0
    0    0-0.02 0.27 0.75 0.84-0.14 0.14-0.13 0.09 0.62 0.86 0.49 0.86  0.4 0.77 0.56 0.57    0    0
    0    0 0.13 0.23 0.42 0.63 0.77 0.99 0.45  0.7 0.25 0.42 0.34 0.44 0.25 0.36 0.43 0.42    0    0
    0    0-0.14-0.0430.075 0.29 0.63 0.85 0.35 0.61-0.02 0.16-0.06 0.040.033 0.140.0940.07